In [616]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [528]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Plot settings

In [701]:
HIGHLIGHT_COLOR = "#df2935"
COLORS = ["#007C77", "#1a1b41", "#f48668", "#f2bb05", "#E2A0FF", "#8acb88"]

In [702]:
pio.templates["created_by"] = go.layout.Template(
    layout=go.Layout(
        #font=dict(family="Open Sans"), # Can update to our own stylings when we have it
        title_x=0.05,
        yaxis=dict(tickformat=",.0f"),
        colorway = COLORS,
        #colorway = ["#007C77", "#1a1b41", "#f48668", "#246a73", "#f2bb05", "#E65F5C", "#8acb88"],
        #colorway=["#127475", "#1a1b41", "#f48668", "#246a73", "#f2bb05", "#8acb88"],
        hovermode='x unified',
    ),
    layout_annotations=[
        dict(
            name="created by",
            text="Created by Electrify RVA",
            #textangle=-30,
            opacity=0.9,
            font=dict(color="black", size=12),
            xref="paper",
            yref="paper",
            x=-0.05,
            y=-0.25,
            showarrow=False,
        )
    ]
)

pio.templates.default = "ggplot2+created_by"

# Functions

In [632]:
# Converters for reading in data types that confuse pandas
def percent_to_float(x):
    return float(x.strip('%')) if len(x) > 0 else np.nan
    

def remove_number_formatting(x):
    return float(x.replace('$', '').replace(',', '')) if len(x) > 0 else np.nan

In [633]:
lost_gas_yearly = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/1bw4xAAT9WdzBOzTk0tkQIVHCqBT2NImmIu_aP7v0_cY/export?gid=1762730245&format=csv",
    converters={
        'Yearly Loss Rate': percent_to_float,
        'SUM of Total Loss (Mcf)': remove_number_formatting,
        'SUM of Receipt (Mcf)': remove_number_formatting,
        'EIA Citygate Price $/MCF': remove_number_formatting,
        '$ Lost to Leak': remove_number_formatting,
    },
    skipfooter=1,
).drop(['Unnamed: 3', 'Unnamed: 4'], axis=1)

lost_gas_yearly

/var/folders/p0/54kr448x6679fjp6vzs43l440000gn/T/ipykernel_47275/1502407349.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.



,Month - Year,SUM of Total Loss (Mcf),SUM of Receipt (Mcf),Yearly Loss Rate,EIA Citygate Price $/MCF,$ Lost to Leak
0,2012,475204.0,24709147.0,1.9,5.64,2680151.0
1,2013,383433.0,19498466.0,2.0,5.54,2124219.0
2,2014,471739.0,19851432.0,2.4,5.98,2820999.0
3,2015,574615.0,27187493.0,2.1,4.87,2798375.0
4,2016,626795.0,24692644.0,2.5,3.99,2500912.0
5,2017,419827.0,17996974.0,2.3,4.53,1901816.0
6,2018,1004674.0,18386512.0,5.5,4.69,4711921.0
7,2019,901223.0,18916623.0,4.8,4.52,4073528.0
8,2020,575102.0,21506914.0,2.7,3.87,2225645.0
9,2021,1029281.0,22797084.0,4.5,5.07,5218455.0


In [639]:

lost_gas_monthly = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/1bw4xAAT9WdzBOzTk0tkQIVHCqBT2NImmIu_aP7v0_cY/export?gid=983926144&format=csv",
    converters={
        'Loss %': percent_to_float,
        '12 month % LUF': percent_to_float,
        'Receipt (Mcf)': remove_number_formatting,
        'Total Loss (Mcf)': remove_number_formatting,
        '12 month total LUF (Mcf)': remove_number_formatting,
        '12 month total receipts (Mcf)': remove_number_formatting,
        'Tipping Point': percent_to_float,
    },
    parse_dates=['Month'],
)

lost_gas_monthly.head()

,Month,Receipt (Mcf),Total Loss (Mcf),Loss %,12 month total LUF (Mcf),12 month total receipts (Mcf),12 month % LUF,Loss > Lower Bound,Loss > Upper bound,12 mo loss > Lower Bound,12 mo loss > Upper bound,Tipping Point
0,2012-01-01,2450103.0,185548.0,7.6,NaN,NaN,NaN,1,1,NaN,NaN,3.2
1,2012-02-01,2112200.0,62730.0,3.0,NaN,NaN,NaN,0,0,NaN,NaN,3.2
2,2012-03-01,1216334.0,1013.0,0.1,NaN,NaN,NaN,0,0,NaN,NaN,3.2
3,2012-04-01,1724348.0,65866.0,3.8,NaN,NaN,NaN,1,0,NaN,NaN,3.2
4,2012-05-01,2117391.0,-42936.0,-2.0,NaN,NaN,NaN,0,0,NaN,NaN,3.2


In [534]:
eia_phmsa = pd.read_csv("data/rgw_eia_phmsa.csv")

In [535]:
eia_phmsa[[col for col in eia_phmsa.columns if "residential" in col]]

,residential_any,residential_all,residential_last_11,residential_total_volume_mcf,residential_sales_volume_mcf,residential_transported_volume_mcf,residential_sales_price_dollars,residential_transported_price_dollars,residential_sales_revenue_dollars,residential_transported_revenue_dollars,residential_total_customers,residential_sales_customers,residential_transported_customers,residential_volume_per_customer,residential_sales_per_customer,residential_sales_price_dollars_real,residential_transported_price_dollars_real,residential_sales_revenue_dollars_real,residential_transported_revenue_dollars_real,residential_sales_per_customer_real,residential_cost_per_mcf_real,residential_sales_customers_growth,residential_sales_volume_mcf_growth,residential_sales_revenue_dollars_real_growth,residential_fuel_costs_real,residential_net_revenue_real,residential_net_revenue_real_per_customer,residential_net_revenue_real_proportion,mains_miles_per_residential_customers
0,True,True,True,9446820,9446820,0,6.61,0,62459939,0,78195,78195,0,120.811049,798.771520,4.150453,0,3.921892e+07,0,501.552791,4.151547,NaN,NaN,NaN,2.455727e+07,1.466165e+07,187.501143,0.577056,0.020052
1,True,True,True,5988272,5988272,0,8.66,0,51876780,0,79059,79059,0,75.744343,656.178044,5.521780,0,3.307762e+07,0,418.391523,5.523733,864.0,-3458548.0,-6.141305e+06,1.428020e+07,1.879741e+07,237.764391,0.619644,0.020402
2,True,True,True,6378840,6378840,0,9.02,0,57549141,0,81523,81523,0,78.245894,705.925211,5.877457,0,3.749918e+07,0,459.982844,5.878683,2464.0,390568.0,4.421566e+06,1.583615e+07,2.166303e+07,265.729067,0.701357,0.020252
3,True,True,True,7421796,7421796,0,9.67,0,71781543,0,81531,81531,0,91.030357,880.420245,6.513134,0,4.834775e+07,0,592.998428,6.514293,8.0,1042956.0,1.084857e+07,2.669401e+07,2.165375e+07,265.589120,0.754695,0.020569
4,True,True,True,6538829,6538829,0,10.83,0,70842491,0,84150,84150,0,77.704444,841.859667,7.499897,0,4.905923e+07,0,582.997333,7.502754,2619.0,-882967.0,7.114707e+05,3.061072e+07,1.844851e+07,219.233581,0.976618,0.020107
5,True,True,True,6663028,6663028,0,10.99,0,73246397,0,87322,87322,0,76.304116,838.808055,7.732141,0,5.153334e+07,0,590.153054,7.734223,3172.0,124199.0,2.474119e+06,3.164299e+07,1.989036e+07,227.781746,0.916312,0.019846
6,True,True,True,7616351,7616351,0,11.58,0,88198579,0,87322,87322,0,87.221445,1010.038467,8.334465,0,6.347910e+07,0,726.954279,8.334582,0.0,953323.0,1.194576e+07,3.601484e+07,2.746426e+07,314.517080,0.810005,0.020224
7,True,True,True,7209134,7209134,0,14.34,0,103344199,0,90274,90274,0,79.858365,1144.783648,10.596236,0,7.636398e+07,0,845.913378,10.592671,2952.0,-407217.0,1.288488e+07,4.053874e+07,3.582525e+07,396.850097,0.630355,0.019784
8,True,True,True,7355609,7355609,0,16.16,0,118853222,0,91543,91543,0,80.351409,1298.332172,12.343003,0,9.078006e+07,0,991.665723,12.341610,1269.0,146475.0,1.441607e+07,5.685630e+07,3.392375e+07,370.577219,0.656805,0.019892
9,True,True,True,5976557,5976557,0,16.73,0,99965290,0,93584,93584,0,63.863021,1068.187831,13.190099,0,7.881363e+07,0,842.169968,13.187130,2041.0,-1379052.0,-1.196642e+07,4.952288e+07,2.929075e+07,312.988896,0.709085,0.019505


# Plots

## Plotting functions

In [536]:
def lineplot_eia_phmsa(y:str, x:str="year", title:str="", x_label="", y_label="", filename:str=None) -> None:
    fig = px.line(
        eia_phmsa, 
        x=x, y=y, 
        title=title,
        labels={ y: y_label, x: x_label, }
    )

    fig.update_traces(hovertemplate=None)

    filename = filename or y

    fig.show()
    fig.write_html(f"../docs/_includes/charts/{filename}.html", include_plotlyjs="cdn")

## EIA & PHMSA

In [537]:
fig = px.line(
    eia_phmsa, 
    x="year", y="mains_miles_total", 
    title='Miles of pipelines',
    labels={ "year": "", "mains_miles_total": "Pipelines", }
)

fig.update_layout(yaxis=dict(tickformat=",.0f"))
fig.show()
fig.write_html("../docs/_includes/charts/pipeline_miles.html", include_plotlyjs="cdn")

In [538]:
fig = px.line(
    eia_phmsa, 
    x="year", y="residential_sales_volume_mcf", 
    title='Residential Methane Sales',
    labels={ "year": "", "residential_sales_volume_mcf": "Total methane sold (Mcf)", }
)
fig.show()
fig.write_html("../docs/_includes/charts/residential_sales.html", include_plotlyjs="cdn")


In [539]:
customer_variables = {
    "residential_total_customers": "Residential", 
    "commercial_total_customers": "Commercial", 
    "industrial_total_customers": "Industrial", 
    "electric_total_customers": "Electric",
    "vehicle_fuel_total_customers": "Vehicle Fuel",
}

melted = eia_phmsa[["year", *[*customer_variables]]].melt(id_vars="year", var_name="type_unformatted", value_name="Customers")
melted["Type"] = melted.apply(lambda r: customer_variables[r['type_unformatted']], axis=1)
melted

fig = px.bar(
    melted, x="year", y="Customers", title="Customers", color="Type",
    labels={ "year": "", }
)

fig.update_traces(hovertemplate=None)
fig.update_layout(yaxis=dict(tickformat=",.0f"), hovermode='x unified')

fig.show()
fig.write_html("../docs/_includes/charts/customer_types.html", include_plotlyjs="cdn")

In [540]:
sales_variables = {
    "residential_sales_volume_mcf": "Residential", 
    "commercial_sales_volume_mcf": "Commercial", 
    "industrial_sales_volume_mcf": "Industrial", 
    "electric_sales_volume_mcf": "Electric",
    "vehicle_fuel_sales_volume_mcf": "Vehicle Fuel",
}

melted = eia_phmsa[["year", *[*sales_variables]]].melt(id_vars="year", var_name="type_unformatted", value_name="Sales (Mcf)")
melted["Type"] = melted.apply(lambda r: sales_variables[r['type_unformatted']], axis=1)
melted

fig = px.bar(
    melted, x="year", y="Sales (Mcf)", title="Gas sales (in thousands of cubic feet)", color="Type",
    labels={ "year": "", }
)

fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified')

fig.show()
fig.write_html("../docs/_includes/charts/sales_types.html", include_plotlyjs="cdn")


In [541]:
fig = px.line(
    eia_phmsa, 
    x="year", y="residential_sales_customers", 
    title='Residential Customers',
    labels={ "year": "", "residential_sales_customers": "Customers", }
)
fig.add_trace(go.Scatter(
    x=[eia_phmsa['year'].iloc[-1]],
    y=[eia_phmsa['residential_sales_customers'].iloc[-1]],
    mode='markers',
    marker=dict(color=HIGHLIGHT_COLOR, size=7.5),
    showlegend=False,
    hoverinfo='skip',
))
fig.add_annotation(
    text="36% increase<br>since 2000", x=2022, y=111500,
    showarrow=True, arrowhead=2,
    align="right",
)

fig.show()
fig.write_html("../docs/_includes/charts/residential_customers.html", include_plotlyjs="cdn")

In [542]:
# residential_cost_per_mcf_real
# residential_net_revenue_real_per_customer
# residential_net_revenue_real_proportion

In [543]:
lineplot_eia_phmsa(y="residential_cost_per_mcf_real", x_label="", y_label="$ per thousand cubic feet", title="Residential cost per unit of methane")

In [544]:
lineplot_eia_phmsa(y="residential_net_revenue_real_per_customer", x_label="", y_label="$ per Customer", title="Residential system costs per customer")

In [545]:
#lineplot_eia_phmsa(y="residential_volume_per_customer", x_label="", y_label="Units of gas per customer (Mcf)", title="Residential average gas use")
fig = px.line(
    eia_phmsa, 
    x="year", y="residential_volume_per_customer", 
    title='Residential average gas use',
    labels={ "year": "", "residential_volume_per_customer": "Customer average (Mcf)", }
)

fig.add_trace(go.Scatter(
    x=[eia_phmsa['year'].iloc[-1]],
    y=[eia_phmsa['residential_volume_per_customer'].iloc[-1]],
    mode='markers',
    marker=dict(color=HIGHLIGHT_COLOR, size=7.5),
    showlegend=False,
    hoverinfo='skip',
))

fig.add_annotation(
    text="36% decrease<br>since 2000", x=2022, y=59,
    showarrow=True, arrowhead=2,
    align="right",
)

fig.update_traces(hovertemplate=None)

fig.show()
fig.write_html("../docs/_includes/charts/residential_volume_per_customer.html", include_plotlyjs="cdn")

In [546]:
#lineplot_eia_phmsa(y="residential_fuel_costs_real", x_label="", y_label="Total cost of gas ($)", title="Residential fuel costs")

costs_variables = {
    "residential_net_revenue_real": "System", 
    "residential_fuel_costs_real": "Fuel", 
}

melted = eia_phmsa[["year", *[*costs_variables]]].melt(id_vars="year", var_name="type_unformatted", value_name="Costs")
melted["Cost source"] = melted.apply(lambda r: costs_variables[r['type_unformatted']], axis=1)
melted

fig = px.area(
    melted, x="year", y="Costs", title="Residential system and fuel costs", color="Cost source",
    labels={ "year": "", },
    color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_layout(yaxis_tickprefix = '$', hovermode='x unified')


fig.show()
fig.write_html("../docs/_includes/charts/system_and_fuel_costs.html", include_plotlyjs="cdn")

In [547]:
eia_phmsa['residential_net_revenue_real_percent'] = eia_phmsa['residential_net_revenue_real'] / eia_phmsa['residential_sales_revenue_dollars_real'] * 100
eia_phmsa['residential_fuel_costs_real_percent'] = eia_phmsa['residential_fuel_costs_real'] / eia_phmsa['residential_sales_revenue_dollars_real'] * 100

costs_variables = {
    "residential_net_revenue_real_percent": "System", 
    "residential_fuel_costs_real_percent": "Fuel", 
}

melted = eia_phmsa[["year", *[*costs_variables]]].melt(id_vars="year", var_name="type_unformatted", value_name="Costs")
melted["Cost source"] = melted.apply(lambda r: costs_variables[r['type_unformatted']], axis=1)
melted

fig = px.area(
    melted, x="year", y="Costs", title="Residential system and fuel costs", color="Cost source",
    labels={ "year": "", },
    color_discrete_sequence=["#1a1b41", "#f48668"],
    #groupnorm='percent',
)

fig.add_hline(y=50, line_color="white", line_width=1)

fig.update_traces(hovertemplate=None)
fig.update_layout(
    showlegend=True,
    #xaxis_type='category',
    yaxis=dict(
        type='linear',
        range=[0, 100],
        ticksuffix='%',
        tickprefix='',
    ),
    hovermode='x unified',
)
fig.add_annotation(
    text="50%", x=1, y=50,
    showarrow=False, arrowhead=2,
    valign='bottom', align="right",
    font=dict(color='white'),
    xref='paper',
)

fig.show()
fig.write_html("../docs/_includes/charts/system_and_fuel_costs_normalized.html", include_plotlyjs="cdn")


In [548]:
# lineplot_eia_phmsa(y="residential_net_revenue_real_per_customer", x_label="", y_label="$ per Customer", title="Residential system costs per customer")

fig = px.line(
    eia_phmsa, 
    x="year", y="residential_net_revenue_real_per_customer", 
    title='Residential annual system costs per customer',
    labels={ "year": "", "residential_net_revenue_real_per_customer": "Annual system costs", },
    color_discrete_sequence=["#1a1b41", "#f48668"],

)
fig.add_trace(go.Scatter(
    x=[eia_phmsa['year'].iloc[-1]],
    y=[eia_phmsa['residential_net_revenue_real_per_customer'].iloc[-1]],
    mode='markers',
    marker=dict(color=HIGHLIGHT_COLOR, size=7.5),
    showlegend=False,
    hoverinfo='skip',
))

fig.update_traces(hovertemplate=None)
fig.update_layout(yaxis_tickprefix = '$', hovermode='x unified',)
fig.add_annotation(
    text="$573", x=2022, y=575,
    showarrow=False, arrowhead=2,
    yshift=11,
    font=dict(color=HIGHLIGHT_COLOR),
)

fig.show()
fig.write_html("../docs/_includes/charts/residential_net_revenue_real_per_customer.html", include_plotlyjs="cdn")

In [549]:
#lineplot_eia_phmsa(y="residential_sales_revenue_dollars_real", x_label="", y_label="Sales revenue", title="Residential sales revenue")

fig = px.line(
    eia_phmsa, 
    x="year", y="residential_sales_revenue_dollars_real", 
    title='Residential total sales revenue',
    labels={ "year": "", "residential_sales_revenue_dollars_real": "Total sales revenue (2019$)", },
    #color_discrete_sequence=["#1a1b41", "#f48668"],

)
fig.add_trace(go.Scatter(
    x=[eia_phmsa['year'].iloc[-1]],
    y=[eia_phmsa['residential_sales_revenue_dollars_real'].iloc[-1]],
    mode='markers',
    marker=dict(color=HIGHLIGHT_COLOR, size=7.5),
    showlegend=False,
    hoverinfo='skip',
))

fig.update_traces(hovertemplate=None)
fig.update_layout(yaxis_tickprefix = '$', hovermode='x unified',)
fig.add_annotation(
         text="$118.5M", x=2022, y=118537794,
         showarrow=False, arrowhead=2,
         yshift=11,
         font=dict(color=HIGHLIGHT_COLOR),
)

fig.show()
fig.write_html("../docs/_includes/charts/residential_sales_revenue_dollars_real.html", include_plotlyjs="cdn")

In [550]:
eia_phmsa[["year", *[col for col in eia_phmsa.columns if "mains" in col]]]

,year,mains_miles_steel_unprotected_bare,mains_miles_steel_unprotected_coated,mains_miles_steel_cathodically_protected_bare,mains_miles_steel_cathodically_protected_coated,mains_miles_plastic,mains_miles_cast_iron,mains_miles_ductile_iron,mains_miles_copper,mains_miles_other1_pre2010,mains_miles_other2_pre2010,mains_miles_total,mains_miles_steel_unknown,mains_miles_steel_lt2in,mains_miles_steel_2in_to_4in,mains_miles_steel_4in_to_8in,mains_miles_steel_8in_to_12in,mains_miles_steel_gt12in,mains_miles_steel_total,mains_miles_ductile_iron_unknown,mains_miles_ductile_iron_lt2in,mains_miles_ductile_iron_2in_to_4in,mains_miles_ductile_iron_4in_to_8in,mains_miles_ductile_iron_8in_to_12in,mains_miles_ductile_iron_gt12in,mains_miles_ductile_iron_total,mains_miles_copper_unknown,mains_miles_copper_lt2in,mains_miles_copper_2in_to_4in,mains_miles_copper_4in_to_8in,mains_miles_copper_8in_to_12in,mains_miles_copper_gt12in,mains_miles_copper_total,mains_miles_cast_iron_wr_unknown,mains_miles_cast_iron_wr_lt2in,mains_miles_cast_iron_wr_2in_to_4in,mains_miles_cast_iron_wr_4in_to_8in,mains_miles_cast_iron_wr_8in_to_12in,mains_miles_cast_iron_wr_gt12in,mains_miles_cast_iron_wr_total,mains_miles_plastic_pvc_unknown,mains_miles_plastic_pvc_lt2in,mains_miles_plastic_pvc_2in_to_4in,mains_miles_plastic_pvc_4in_to_8in,mains_miles_plastic_pvc_8in_to_12in,mains_miles_plastic_pvc_gt12in,mains_miles_plastic_pvc_total,mains_miles_plastic_pe_unknown,mains_miles_plastic_pe_lt2in,mains_miles_plastic_pe_2in_to_4in,mains_miles_plastic_pe_4in_to_8in,mains_miles_plastic_pe_8in_to_12in,mains_miles_plastic_pe_gt12in,mains_miles_plastic_pe_total,mains_miles_plastic_abs_unknown,mains_miles_plastic_abs_lt2in,mains_miles_plastic_abs_2in_to_4in,mains_miles_plastic_abs_4in_to_8in,mains_miles_plastic_abs_8in_to_12in,mains_miles_plastic_abs_gt12in,mains_miles_plastic_abs_total,mains_miles_other1_unknown,mains_miles_other1_lt2in,mains_miles_other1_2in_to_4in,mains_miles_other1_4in_to_8in,mains_miles_other1_8in_to_12in,mains_miles_other1_gt12in,mains_miles_other1_total,mains_miles_other2_unknown,mains_miles_other2_lt2in,mains_miles_other2_2in_to_4in,mains_miles_other2_4in_to_8in,mains_miles_other2_8in_to_12in,mains_miles_other2_gt12in,mains_miles_other2_total,mains_miles_unknown_total,mains_miles_lt2in_total,mains_miles_2in_to_4in_total,mains_miles_4in_to_8in_total,mains_miles_8in_to_12in_total,mains_miles_gt12in_total,mains_miles_part_b2_total,total_leaks_corrosion_failure_mains,total_leaks_third_party_damage_mains_pre2004,total_leaks_outside_force_damage_mains_pre2004,total_leaks_construction_damage_mains_pre2004,total_leaks_material_defect_mains_pre2004,total_leaks_other_mains,mains_miles_by_decade_unknown,mains_miles_by_decade_pre1940,mains_miles_by_decade_1940_to_1949,mains_miles_by_decade_1950_to_1959,mains_miles_by_decade_1960_to_1969,mains_miles_by_decade_1970_to_1979,mains_miles_by_decade_1980_to_1989,mains_miles_by_decade_1990_to_1999,mains_miles_by_decade_2000_to_2009,mains_miles_by_decade_total,total_leaks_natural_force_damage_mains,total_leaks_excavation_damage_mains,total_leaks_outside_force_damage_mains,total_leaks_pipe_weld_joint_failure_mains,total_leaks_equipment_failure_mains,total_leaks_incorrect_operation_mains,mains_miles_other,mains_miles_plastic_other_unknown,mains_miles_plastic_other_lt2in,mains_miles_plastic_other_2in_to_4in,mains_miles_plastic_other_4in_to_8in,mains_miles_plastic_other_8in_to_12in,mains_miles_plastic_other_gt12in,mains_miles_plastic_other_total,mains_miles_other_unknown,mains_miles_other_lt2in,mains_miles_other_2in_to_4in,mains_miles_other_4in_to_8in,mains_miles_other_8in_to_12in,mains_miles_other_gt12in,mains_miles_other_total,mains_miles_by_decade_2010_to_2019,mains_miles_by_decade_2020_to_2029,total_hazardous_leaks_corrosion_failure_mains,total_hazardous_leaks_natural_force_damage_mains,total_hazardous_leaks_excavation_damage_mains,total_hazardous_leaks_outside_force_damage_mains,total_hazardous_leaks_pipe_weld_joint_failure_mai

## Pipeline plots

In [551]:
# mains_miles_total
# services_miles_total
# mains_services_miles_total
pipe_variables = {
    "mains_miles_total": "Mains", 
    "services_miles_total": "Services", 
}

melted = eia_phmsa[["year", *[*pipe_variables]]].melt(id_vars="year", var_name="type_unformatted", value_name="Miles")
melted["Pipe"] = melted.apply(lambda r: pipe_variables[r['type_unformatted']], axis=1)
melted

fig = px.area(
    melted, x="year", y="Miles", title="Miles of pipes", color="Pipe",
    labels={ "year": "", },
    color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified')


fig.show()
fig.write_html("../docs/_includes/charts/total_miles_by_type.html", include_plotlyjs="cdn")


In [552]:
[col for col in eia_phmsa.columns if "services_miles" in col]

['services_miles_leak_prone_pipes',
 'mains_services_miles_leak_prone_pipes',
 'services_miles_total',
 'mains_services_miles_total',
 'services_miles_proportion',
 'services_miles_change']

In [553]:
# Copper and Other are basically non-existent for RGW mains but do exist for services
materials = pd.DataFrame(
    [
        ('mains_miles_steel_unprotected_bare', 'number_services_steel_unprotected_bare', 'Unprotected, Bare', 'Steel', 'Leak prone'),
        ('mains_miles_steel_unprotected_coated', 'number_services_steel_unprotected_coated', 'Unprotected, Coated', 'Steel', 'Leak prone'),
        ('mains_miles_steel_cathodically_protected_bare', 'number_services_steel_cathodically_protected_bare', 'Cathodically protected, Bare', 'Steel', 'Leak prone'),
        ('mains_miles_steel_cathodically_protected_coated', 'number_services_steel_cathodically_protected_coated', 'Cathodically protected, Coated', 'Steel', 'Non-Leak prone'),
        ('mains_miles_cast_iron', 'number_services_cast_iron', 'Cast Iron', 'Iron', 'Leak prone'),
        ('mains_miles_ductile_iron', 'number_services_ductile_iron', 'Ductile Iron', 'Iron', 'Leak prone'),
        ('mains_miles_plastic', 'number_services_plastic', 'Plastic', 'Plastic', 'Non-Leak prone'),
        ('mains_miles_copper', 'number_services_copper', 'Copper', 'Copper', 'Leak prone'),
        ('mains_miles_other', 'number_services_other', 'Other', 'Other', 'Leak prone'),
    ],
    columns=["mains_variable", "services_variable", "material_specific", "material", "leak_prone"]
)


In [554]:
mains_melted = eia_phmsa[
    ["year", *materials['mains_variable'].to_list()]
].melt(
    id_vars="year",
    var_name="mains_variable", value_name="Mains miles"
).merge(
    materials, on=['mains_variable']
).drop('services_variable', axis=1)

services_melted = eia_phmsa[
    ["year", *materials['services_variable'].to_list()]
].melt(
    id_vars="year",
    var_name="services_variable", value_name="Number of services"
).merge(
    materials, on=['services_variable']
).drop('mains_variable', axis=1)

melted = mains_melted.merge(
    services_melted, on=['year', 'material_specific', 'material', 'leak_prone']
).merge(
    eia_phmsa[['year', 'average_service_length']]
)

# Convert length in feet to miles
melted['Services miles'] = (melted['average_service_length'] / 5280) * melted['Number of services']

melted.head()

,year,mains_variable,Mains miles,material_specific,material,leak_prone,services_variable,Number of services,average_service_length,Services miles
0,1997,mains_miles_steel_unprotected_bare,2.0,"Unprotected, Bare",Steel,Leak prone,number_services_steel_unprotected_bare,4081,100,77.291667
1,1997,mains_miles_steel_unprotected_coated,21.0,"Unprotected, Coated",Steel,Leak prone,number_services_steel_unprotected_coated,9150,100,173.295455
2,1997,mains_miles_steel_cathodically_protected_bare,0.0,"Cathodically protected, Bare",Steel,Leak prone,number_services_steel_cathodically_protected_bare,0,100,0.000000
3,1997,mains_miles_steel_cathodically_protected_coated,440.0,"Cathodically protected, Coated",Steel,Non-Leak prone,number_services_steel_cathodically_protected_c...,5163,100,97.784091
4,1997,mains_miles_cast_iron,556.0,Cast Iron,Iron,Leak prone,number_services_cast_iron,245,100,4.640152


In [555]:
fig = px.area(
    melted, x="year", y="Mains miles", title="Mains miles by material", color="material", line_group='material_specific',
    labels={ "year": "", "material": "Material"},
    hover_data=["material_specific"],
    #color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')


fig.show()
fig.write_html("../docs/_includes/charts/material_mains_miles.html", include_plotlyjs="cdn")

In [556]:
fig = px.area(
    melted, x="year", y="Services miles", title="Services miles by material", color="material", line_group='material_specific',
    labels={ "year": "", "material": "Material"},
    hover_data=["material_specific"],
    #color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')


fig.show()
fig.write_html("../docs/_includes/charts/material_services_miles.html", include_plotlyjs="cdn")

In [557]:
fig = px.area(
    melted, x="year", y="Mains miles", title="Leak prone mains miles", color="leak_prone", line_group='material_specific',
    labels={ "year": "", "material": "Material", "leak_prone": "Leak prone"},
    hover_data=["material_specific"],
    color_discrete_map={'Leak prone': HIGHLIGHT_COLOR}
    #color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')

fig.add_annotation(
    text="196 miles", x=1.075, y=196,
    showarrow=False, arrowhead=2,
    xref='paper',
    font=dict(color=HIGHLIGHT_COLOR),
)


fig.show()
fig.write_html("../docs/_includes/charts/material_leakprone_mains_miles.html", include_plotlyjs="cdn")

In [558]:
fig = px.area(
    melted, x="year", y="Services miles", title="Leak prone services miles", color="leak_prone", line_group='material_specific',
    labels={ "year": "", "material": "Material", "leak_prone": "Leak prone"},
    hover_data=["material_specific"],
    color_discrete_map={'Leak prone': HIGHLIGHT_COLOR}
    #color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')

fig.add_annotation(
    text="118 miles", x=1.075, y=118,
    showarrow=False, arrowhead=2,
    xref='paper',
    font=dict(color=HIGHLIGHT_COLOR),
)


fig.show()
fig.write_html("../docs/_includes/charts/material_leakprone_services_miles.html", include_plotlyjs="cdn")

In [559]:
melted_all = melted.melt(
    id_vars=['year', 'material', 'material_specific', 'leak_prone'],
    value_vars=['Mains miles', 'Services miles'],
    var_name='pipe_type', value_name='Miles',
)

melted_all['Pipe'] = melted_all['pipe_type'].str.split(" ").str.get(0)

melted_all.head()

,year,material,material_specific,leak_prone,pipe_type,Miles,Pipe
0,1997,Steel,"Unprotected, Bare",Leak prone,Mains miles,2.0,Mains
1,1997,Steel,"Unprotected, Coated",Leak prone,Mains miles,21.0,Mains
2,1997,Steel,"Cathodically protected, Bare",Leak prone,Mains miles,0.0,Mains
3,1997,Steel,"Cathodically protected, Coated",Non-Leak prone,Mains miles,440.0,Mains
4,1997,Iron,Cast Iron,Leak prone,Mains miles,556.0,Mains


In [560]:
fig = px.area(
    melted_all, 
    x="year", y="Miles", color="material", line_group='material_specific', pattern_shape='Pipe',
    title="Miles of pipes", 
    labels={ "year": "", "material": "Material"},
    pattern_shape_sequence=["", "\\"],
    hover_data=["material_specific", "Pipe"],
    #color_discrete_sequence=["#1a1b41", "#f48668"],
)

fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')

fig.show()
fig.write_html("../docs/_includes/charts/material_mains_services_miles.html", include_plotlyjs="cdn")

In [566]:
fig = px.area(
    melted_all, 
    x="year", y="Miles", color="leak_prone", line_group='material_specific', pattern_shape='Pipe',
    title="Miles of pipes", 
    labels={ "year": "", "material": "Material", "leak_prone": "Leak prone"},
    pattern_shape_sequence=["", "\\"],
    hover_data=["material_specific", "Pipe"],
    color_discrete_map={'Leak prone': HIGHLIGHT_COLOR},
)

fig.add_annotation(
    text="314 miles", x=1.1, y=314,
    showarrow=False, arrowhead=2,
    xref='paper',
    font=dict(color=HIGHLIGHT_COLOR),
)

fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')
fig.update_traces()

fig.show()
fig.write_html("../docs/_includes/charts/material_leakprone_mains_services_miles.html", include_plotlyjs="cdn")

# By decade

In [562]:
eia_phmsa[["year", *[col for col in eia_phmsa.columns if "decade" in col]]]

,year,mains_miles_by_decade_unknown,mains_miles_by_decade_pre1940,mains_miles_by_decade_1940_to_1949,mains_miles_by_decade_1950_to_1959,mains_miles_by_decade_1960_to_1969,mains_miles_by_decade_1970_to_1979,mains_miles_by_decade_1980_to_1989,mains_miles_by_decade_1990_to_1999,mains_miles_by_decade_2000_to_2009,mains_miles_by_decade_total,number_services_by_decade_unknown,number_services_by_decade_pre1940,number_services_by_decade_1940_to_1949,number_services_by_decade_1950_to_1959,number_services_by_decade_1960_to_1969,number_services_by_decade_1970_to_1979,number_services_by_decade_1980_to_1989,number_services_by_decade_1990_to_1999,number_services_by_decade_2000_to_2009,number_services_by_decade_total,mains_miles_by_decade_2010_to_2019,mains_miles_by_decade_2020_to_2029,number_services_by_decade_2010_to_2019,number_services_by_decade_2020_to_2029
0,1997,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
1,1998,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
2,1999,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
3,2000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
4,2001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
5,2002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
6,2003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0,0
7,2004,0.000,218.000,64.000,163.000,171.000,145.000,470.000,427.000,128.000,1786.000,281,10117,5995,8431,8806,7588,19017,20890,12553,93678,0.000,0.000,0,0
8,2005,0.000,207.000,64.000,162.000,171.000,145.000,470.000,427.000,175.000,1821.000,281,9365,5995,8431,8806,7588,19017,20890,15644,96017,0.000,0.000,0,0
9,2006,0.000,131.000,44.000,137.000,159.000,145.000,470.000,427.000,312.000,1825.000,0,7761,5266,7848,8806,7588,19017,20890,20885,98061,0.000,0.000,0,0


In [567]:
decade_variables_suffix = [
    ("unknown", "Unknown"),
    ("pre1940", "Pre 1940"),
    ("1940_to_1949", "1940s"),
    ("1950_to_1959", "1950s"),
    ("1960_to_1969", "1960s"),
    ("1970_to_1979", "1970s"),
    ("1980_to_1989", "1980s"),
    ("1990_to_1999", "1990s"),
    ("2000_to_2009", "2000s"),
    ("2010_to_2019", "2010s"),
    ("2020_to_2029", "2020s"),
]

decades_variables_mains = pd.DataFrame(
    [(f"mains_miles_by_decade_{decade}", label) for decade, label in decade_variables_suffix], 
    columns=['decade_variable', 'decade'])
decades_variables_services = pd.DataFrame(
    [(f"number_services_by_decade_{decade}", label) for decade, label in decade_variables_suffix], 
    columns=['decade_variable', 'decade'])

In [570]:
melted_decades_mains = eia_phmsa[["year", *decades_variables_mains['decade_variable'].to_list()]].melt(
    id_vars=['year'],
    #value_vars=['Mains miles'],
    var_name='decade_variable', 
    value_name='Mains miles',
).merge(
    decades_variables_mains,
    on='decade_variable'
)

melted_decades_services = eia_phmsa[["year", *decades_variables_services['decade_variable'].to_list()]].melt(
    id_vars=['year'],
    #value_vars=['Mains miles'],
    var_name='decade_variable', 
    value_name='Number of services',
).merge(
    decades_variables_services,
    on='decade_variable'
).merge(
    eia_phmsa[['year', 'average_service_length']]
)

# Convert length in feet to miles
melted_decades_services['Services miles'] = (melted_decades_services['average_service_length'] / 5280) * melted_decades_services['Number of services']


In [589]:
fig = px.area(
    melted_decades_mains, x="year", y="Mains miles", title="Mains miles by decade", color="decade",
    labels={ "year": "", "decade": "Decade"},
    #hover_data=["material_specific"],
    #color_discrete_map={'Unknown': HIGHLIGHT_COLOR},
    color_discrete_sequence=[HIGHLIGHT_COLOR, *px.colors.sequential.Cividis],
)

fig.update_traces(hovertemplate=None)
#fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')
fig.update_xaxes(range=[2004, 2022])



fig.show()
fig.write_html("../docs/_includes/charts/decades_mains_miles.html", include_plotlyjs="cdn")

In [590]:
fig = px.area(
    melted_decades_services, x="year", y="Services miles", title="Services miles by decade", color="decade",
    labels={ "year": "", "decade": "Decade"},
    #hover_data=["material_specific"],
    #color_discrete_map={'Unknown': HIGHLIGHT_COLOR},
    color_discrete_sequence=[HIGHLIGHT_COLOR, *px.colors.sequential.Cividis],
)

fig.update_traces(hovertemplate=None)
#fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')
fig.update_xaxes(range=[2004, 2022])



fig.show()
fig.write_html("../docs/_includes/charts/decades_services_miles.html", include_plotlyjs="cdn")

In [591]:
fig = px.area(
    melted_decades_services, x="year", y="Number of services", title="Number of services by decade", color="decade",
    labels={ "year": "", "decade": "Decade"},
    #hover_data=["material_specific"],
    #color_discrete_map={'Unknown': HIGHLIGHT_COLOR},
    color_discrete_sequence=[HIGHLIGHT_COLOR, *px.colors.sequential.Cividis],
)

fig.update_traces(hovertemplate=None)
#fig.update_traces(hovertemplate = '%{customdata[0]} : %{y}')  #'year=%{customdata[0]}<br>label=%{customdata[1]}<extra></extra>')
fig.update_layout(hovermode='x unified')
fig.update_xaxes(range=[2004, 2022])



fig.show()
fig.write_html("../docs/_includes/charts/decades_services_number.html", include_plotlyjs="cdn")

In [574]:
eia_phmsa['operator_id_phmsa']

0     17360
1     17360
2     17360
3     17360
4     17360
5     17360
6     17360
7     17360
8     17360
9     17360
10    17360
11    17360
12    17360
13    17360
14    17360
15    17360
16    17360
17    17360
18    17360
19    17360
20    17360
21    17360
22    17360
23    17360
24    17360
25    17360
Name: operator_id_phmsa, dtype: int64

## Lost gas
Source: FOIA

In [666]:
lost_gas_yearly

,Month - Year,SUM of Total Loss (Mcf),SUM of Receipt (Mcf),Yearly Loss Rate,EIA Citygate Price $/MCF,$ Lost to Leak
0,2012,475204.0,24709147.0,1.9,5.64,2680151.0
1,2013,383433.0,19498466.0,2.0,5.54,2124219.0
2,2014,471739.0,19851432.0,2.4,5.98,2820999.0
3,2015,574615.0,27187493.0,2.1,4.87,2798375.0
4,2016,626795.0,24692644.0,2.5,3.99,2500912.0
5,2017,419827.0,17996974.0,2.3,4.53,1901816.0
6,2018,1004674.0,18386512.0,5.5,4.69,4711921.0
7,2019,901223.0,18916623.0,4.8,4.52,4073528.0
8,2020,575102.0,21506914.0,2.7,3.87,2225645.0
9,2021,1029281.0,22797084.0,4.5,5.07,5218455.0


In [681]:
fig = px.line(
    lost_gas_monthly, 
    x="Month", y="12 month % LUF", 
    title='Lost and unaccounted for gas',
    labels={ "Month": "", "12 month % LUF": "Loss rate (12 month average)", },
    #color_discrete_sequence=["#1a1b41", "#f48668"],

)
fig.add_trace(go.Scatter(
    x=[lost_gas_monthly['Month'].iloc[-1]],
    y=[lost_gas_monthly['12 month % LUF'].iloc[-1]],
    mode='markers',
    marker=dict(color=HIGHLIGHT_COLOR, size=7.5),
    showlegend=False,
    hoverinfo='skip',
))

fig.update_traces(hovertemplate=None)
fig.update_layout(yaxis_ticksuffix='%', hovermode='x unified', yaxis_tickformat=",.1f")

fig.add_annotation(
         text="9.1%", x='2022-12-01', y=9.1,
         showarrow=False, arrowhead=2,
         yshift=11,
         font=dict(color=HIGHLIGHT_COLOR),
)
fig.update_xaxes(range=['2012-10-01', '2023-02-01'])

# Leaks >3.2% are worse than coal
worse_than_coal_color = HIGHLIGHT_COLOR
fig.add_hline(
    y=3.2, line_color=worse_than_coal_color, line_width=1, 
    annotation_text="Worse than coal ⬆", 
    annotation_font_color=worse_than_coal_color,
    annotation_x=0,
    annotation_xanchor="left",
)


fig.show()
fig.write_html("../docs/_includes/charts/lost_and_unaccounted_for_gas_monthly.html", include_plotlyjs="cdn")

/Users/erikshilts/opt/anaconda3/envs/electrify-rva/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [680]:
fig = px.line(
    lost_gas_yearly, 
    x="Month - Year", y="Yearly Loss Rate", 
    title='Lost and unaccounted for gas',
    labels={ "Month - Year": "", "Yearly Loss Rate": "Loss rate", },
    #color_discrete_sequence=["#1a1b41", "#f48668"],

)
fig.add_trace(go.Scatter(
    x=[lost_gas_yearly['Month - Year'].iloc[-1]],
    y=[lost_gas_yearly['Yearly Loss Rate'].iloc[-1]],
    mode='markers',
    marker=dict(color=HIGHLIGHT_COLOR, size=7.5),
    showlegend=False,
    hoverinfo='skip',
))

fig.add_annotation(
         text="9.1%", x='2022', y=9.1,
         showarrow=False, arrowhead=2,
         yshift=11,
         font=dict(color=HIGHLIGHT_COLOR),
)

# Leaks >3.2% are worse than coal
worse_than_coal_color = HIGHLIGHT_COLOR
fig.add_hline(
    y=3.2, line_color=worse_than_coal_color, line_width=1, 
    annotation_text="Worse than coal ⬆", 
    annotation_font_color=worse_than_coal_color,
    annotation_x=0,
    annotation_xanchor="left",
)

#fig.update_xaxes(range=['2012-10-01', '2023-02-01'])
fig.update_traces(hovertemplate=None)
fig.update_layout(yaxis_ticksuffix='%', hovermode='x unified', yaxis_tickformat=",.1f")


fig.show()
fig.write_html("../docs/_includes/charts/lost_and_unaccounted_for_gas_yearly.html", include_plotlyjs="cdn")

In [708]:
colors = [COLORS[0], ] * 11
colors[-1] = HIGHLIGHT_COLOR

fig = px.bar(
    lost_gas_yearly, 
    x="Month - Year", y="$ Lost to Leak", 
    title='The high cost of lost gas',
    labels={ "Month - Year": "", "$ Lost to Leak": "Yearly loss", },
    #color_discrete_sequence=["#1a1b41", "#f48668"],

)

fig.add_annotation(
         text="$15.9 million", x='2022', y=15926002,
         showarrow=False, arrowhead=2,
         yshift=11,
         font=dict(color=HIGHLIGHT_COLOR),
)
#fig.add_trace(go.Bar(
#    x=[lost_gas_yearly['Month - Year'].iloc[-1]],
#    y=[lost_gas_yearly['$ Lost to Leak'].iloc[-1]],
#    #mode='markers',
#    marker=dict(color=HIGHLIGHT_COLOR),
#    showlegend=False,
#    #hoverinfo='skip',
#))

fig.update_traces(hovertemplate=None, marker_color=colors)
fig.update_layout(yaxis_tickprefix='$', hovermode='x unified')
fig.update_traces()


fig.show()
fig.write_html("../docs/_includes/charts/lost_and_unaccounted_for_gas_cost_yearly.html", include_plotlyjs="cdn")

In [705]:
colors

['#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#007C77',
 '#df2935']